In [ ]:
# Baseline Torch - 03.65

In [ ]:
def f1():
    # Just to initialize Global Variables
    return
hidden_size = 60

In [ ]:
import time
import random
def set_all_parameters():
    
    global epoc_used
    global mn             # Model Number - For essembling
    global batch_size
    global patience
    global nsplits
    global stop_split
    global emb
    global hidden_size
    global linear_size
    global rate_drop_hidden
    global rate_drop_linear
    global rate_drop_embedding
    global loss
    global act
    global opt
    global sch
    global es_mon
    global es_mode
    global Trainable
    global lr
    global method
    global model_list
    global maxlen
    global max_vocab
    global pretext_proc
    global bstart
    global Full_Train_Validation
    global Weight_Method
    global SEED
    global embed_size
    global embedding_matrix
    global use_pretrained_embedding
    global emb_out_of_vocab
    global include_extra_features
    global build_vocab
    global debug
    global subsampling
    global upsampling
    global split_to_run
    debug =False
    subsampling=0
    upsampling=False

    # Start time - 
    bstart = time.time()
    #Possible embeddings ["Glove","Paragram","Google","Wiki","Combined","Concatenated"]**
    ## Early Stop Monitor and Early Stop Mode
    ## Early Stop Monitor Options es_mon = "val_f1" es_mode="max"  for met=[f1]
    #                             es_mon = "val_acc" es_mode="max" for met=["accuracy"]   
    #                             es_mon = "val_loss"

    ###  4 Splits = Validation Split=0.25%
    ###  5 Splits = Validation Split=0.20 %
    ### 10 Splits = Validation Split=0.10 %
    ### 12 Splits = Validation Split=0.0833 %
    ns=4  # Standar Number of Splits
    ### For Stratified k-fold - Validation -> nsplits=stop_splits
    #model_list.append([model_number, weight, model_eval, embeddings,epochs,patience,batch_size, size hidden, size dense, drop hidden, drop dense, drop spatial, loss,optimizer,metric])

    model_list=[]

    ### Embeddings Gl-Glove, P-Paragram, W-Wiki, Go-Google, Q-Quora
    #model_list.append(["6",1,"X","Combined3",5,1, 512,5,5,0,60,16,0.1,0.1,0.1,'binary_crossentropy','adam',["None"],'val_loss','min'])
    model_list.append(["6",1,"X","Combined3",5,1, 512,5,5,0,60,16,0.1,0.1,0.1,'binary_crossentropy','adam',["clr"],'val_loss','min'])
    #   model_list.append(["0",1,"X","Combined",8,1,1024,4,1,2,60,16,0.1,0.1,0.1,'binary_crossentropy','adam',["est"],'val_acc','max'])
 #   model_list.append(["1",1,"X","Combined",5,2, 512,4,1,3,60,16,0.1,0.1,0.1,'binary_crossentropy','adam2',["clr"],'val_loss','min'])
 #   model_list.append(["3",1,"X","Combined",6,1,1024,4,1,4,60,12,0.1,0.1,0.1,'binary_crossentropy','adam2',["clr"],'val_loss','min'])
   # Weight_Method="Score"
    Weight_Method="Manual"
    ###
    ###  Load Train,Val and Test 
    ###
    maxlen = 70 # Maximum Sequence Size 
    max_vocab = 120000 # Maximum Number of Words in Dictionary
    pretext_proc=True
    SEED = 1029
    print (SEED)
    ###
    build_vocab = "both"   ## Other Values train_only or test_only
    ###  Embeddings
    use_pretrained_embedding = True
    emb_out_of_vocab="specific" # other values:  mean or specific
    Trainable=False  # Embedding Layers trainable(True) or not(False)
    ### Include Meta Features in the final model
    include_extra_features=False
    ### Run Valitation in the Full Train Dataset or Just in Stratified Validation Set
    Full_Train_Validation=False
set_all_parameters()

**Import Libraries**

In [ ]:
from IPython.display import display, HTML
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("../input/embeddings"))

# Any results you write to the current directory are saved as output.
from sklearn.model_selection import train_test_split,StratifiedKFold
from tqdm import tqdm, tqdm_notebook
import math
import re
import random
from datetime import timedelta, datetime
import time


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import colorama
from colorama import Fore

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation


from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten, Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D, Reshape, Concatenate
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from keras.layers import Bidirectional, GlobalMaxPool1D,GlobalMaxPooling1D,GlobalAveragePooling1D ,Conv1D, MaxPooling1D, GRU,CuDNNLSTM,CuDNNGRU, Reshape, MaxPooling1D,AveragePooling1D
from keras.optimizers import RMSprop, SGD, Nadam, Adamax, Adam
from keras import backend as K
from keras.engine.topology import Layer
#from keras import initializations
from keras import initializers, regularizers, constraints
from keras.layers import Conv2D, MaxPool2D
import keras.backend as K
%matplotlib inline

import torch
import torch.nn as nn
import torch.utils.data
import torch as t
import torch.nn.functional as F
from torch.optim.optimizer import Optimizer
from torch import optim

import warnings
warnings.filterwarnings('ignore')

**Seed_Torch**

In [ ]:
def seed_torch(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_torch(SEED)

**Text Functions**

In [ ]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

mispell_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

**Add Extra Features**

In [ ]:

def add_features(df):
    
    df['question_text'] = df['question_text'].progress_apply(lambda x:str(x))
    df['total_length'] = df['question_text'].progress_apply(len)
    df['capitals'] = df['question_text'].progress_apply(lambda comment: sum(1 for c in comment if c.isupper()))
    df['caps_vs_length'] = df.progress_apply(lambda row: float(row['capitals'])/float(row['total_length']),
                                axis=1)
    df['num_words'] = df.question_text.str.count('\S+')
    df['num_unique_words'] = df['question_text'].progress_apply(lambda comment: len(set(w for w in comment.split())))
    df['words_vs_unique'] = df['num_unique_words'] / df['num_words']  

    return df

**Load and Process - Train and Test - Datasets**

In [ ]:
import nltk 
# Define a function to split a review into parsed sentences
def review_to_sentences( review ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = nltk.tokenize.sent_tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( nltk.word_tokenize(raw_sentence))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

def load_and_process(pretext_proc=True):

    global max_vocab
    global subsampling
    global upsampling
    
    tqdm.pandas()
    if debug:
        train_df = pd.read_csv("../input/train.csv")[:10000]
        test_df = pd.read_csv("../input/test.csv")[:2000]
    else:
        train_df = pd.read_csv("../input/train.csv")
        test_df = pd.read_csv("../input/test.csv")
        
    print("Train shape : ",train_df.shape)
    print("Test shape : ",test_df.shape)
    if (subsampling > 0): 
        train_sincere=train_df[  (train_df['target'] == 0) ]
        train_insincere=train_df[  (train_df['target'] == 1) ]
        sincere= int(train_sincere['target'].count())
        insincere= int(train_insincere['target'].count())
        remove_n = int(sincere * subsampling /100)
        if (debug):
            print ("Sincere:",sincere)
            print ("Insincere:",insincere)
        drop_indices = np.random.choice(train_sincere.index, remove_n, replace=False)
        train_sincere= train_sincere.drop(drop_indices)     
        train_df = pd.concat([train_sincere,train_insincere])
    if (upsampling):
        ### 
        train_sincere=train_df[  (train_df['target'] == 0) ]
        train_insincere=train_df[  (train_df['target'] == 1) ]
        sincere= int(train_sincere['target'].count())
        insincere= int(train_insincere['target'].count())
        if (debug):
            remove_sincere = int(sincere)-300
            remove_insincere = int(insincere)-50
        else:
            remove_sincere = int(sincere)-298530
            remove_insincere = int(insincere)-22470
        drop_indices = np.random.choice(train_sincere.index, remove_sincere, replace=False)
        train_sincere= train_sincere.drop(drop_indices)     
        drop_indices = np.random.choice(train_insincere.index, remove_insincere, replace=False)
        train_insincere= train_insincere.drop(drop_indices) 
        add_df = pd.concat([train_sincere,train_insincere])
        add_df.drop(['target'], axis=1)
        test_df= pd.concat([test_df,add_df])
        
    print("Train shape : ",train_df.shape)
    print("Test shape : ",test_df.shape)
      
    if (pretext_proc==True):
        # lower
        train_df["question_text"] = train_df["question_text"].progress_apply(lambda x: x.lower())
        test_df["question_text"] = test_df["question_text"].progress_apply(lambda x: x.lower())

        # Clean the text
        train_df["question_text"] = train_df["question_text"].progress_apply(lambda x: clean_text(x))
        test_df["question_text"] = test_df["question_text"].progress_apply(lambda x: clean_text(x))

        # Clean numbers
        train_df["question_text"] = train_df["question_text"].progress_apply(lambda x: clean_numbers(x))
        test_df["question_text"] = test_df["question_text"].progress_apply(lambda x: clean_numbers(x))

        # Clean speelings
        train_df["question_text"] = train_df["question_text"].progress_apply(lambda x: replace_typical_misspell(x))
        test_df["question_text"] = test_df["question_text"].progress_apply(lambda x: replace_typical_misspell(x))

         # Clean Contractions
        #train["question_text"] = train["question_text"].progress_apply(lambda x: clean_contractions(x,contraction_mapping))
        #test["question_text"] = test["question_text"].progress_apply(lambda x: clean_contractions(x,contraction_mapping))
    
    ## fill up the missing values
    train_X = train_df["question_text"].fillna("_##_").values
    test_X = test_df["question_text"].fillna("_##_").values

    if debug:
        max_vocab=10000

    if load_Quora:

        #
        # Calculate Quoras Word2vrc
        #
        all_questions = list(train_X) + list(test_X) # Initialize an empty list of sentences
        questions=[]
        print ("Parsing questions from All Questions")
        for review in all_questions:
            questions += review_to_sentences(review)

        # Import the built-in logging module and configure it so that Word2Vec 
        # creates nice output messages
        import multiprocessing
        import logging

        logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
            level=logging.INFO)

        # Set values for various parameters
        #sg defines the training algorithm. By default (sg=0), CBOW is used. Otherwise (sg=1), skip-gram is employed
        dimension = 300   # Word vector dimensionality                      
        min_word_count = 1    # Minimum word count                        
        num_workers = 2    # Number of threads to run in parallel
        context = 5       # Context window size                                                                                    
        downsampling = 1e-2   # Downsample setting for frequent words

        # Initialize and train the model (this will take some time)
        from gensim.models import word2vec
        print ("Training model...")
        model = word2vec.Word2Vec(questions, \
                                  sg=1, \
                                  size=dimension, \
                                  window = context,   \
                                  workers=num_workers, \
                                  min_count = min_word_count, \
                                  sample = downsampling)

        model_name = "../quora_clean_300d.bin"
        model.wv.save_word2vec_format(model_name, binary=True)     
        
    ## Tokenize the sentences
    tokenizer = Tokenizer(num_words= max_vocab )
    
    vocab_list= list(train_X)+list(test_X)
    if (build_vocab=="train_only"):
        vocab_list = list(train_X)
    if (build_vocab=="test_only"):
        vocab_list = list(test_X)

    tokenizer.fit_on_texts(vocab_list)
    train_X = tokenizer.texts_to_sequences(train_X)
    test_X = tokenizer.texts_to_sequences(test_X)

    ## Pad the sentences 
    train_X = pad_sequences(train_X, maxlen=maxlen)
    test_X = pad_sequences(test_X, maxlen=maxlen)

    ## Get the target values
    train_y = train_df['target'].values
    


    if (include_extra_features):
            ###################### Add Features ###############################
            train = add_features(train_df)
            test = add_features(test_df)
            
            ###
            ### Just Two Features are selected
            ###
            features = train[['caps_vs_length', 'words_vs_unique']].fillna(0)
            test_features = test[['caps_vs_length', 'words_vs_unique']].fillna(0)

            ss = StandardScaler()
            ss.fit(np.vstack((features, test_features)))
            features = ss.transform(features)
            test_features = ss.transform(test_features)
            if debug:
                print (features.shape)
                print (test_features.shape)
    else:
        features=None
        test_features=None    
    
    #shuffling the data
    np.random.seed(SEED)
    trn_idx = np.random.permutation(len(train_X))

    train_X = train_X[trn_idx]
    train_y = train_y[trn_idx]
    if (include_extra_features):
        features=features[trn_idx]
        
    if (max_vocab==None):
        max_vocab = len(tokenizer.word_index) +1
    print ("Vocab Size:", max_vocab)
    return  train_X, test_X, train_y, test_df, tokenizer.word_index,  features, test_features

### Cyclic CLR
Code taken from https://www.kaggle.com/dannykliu/lstm-with-attention-clr-in-pytorch

In [ ]:
# code inspired from: https://github.com/anandsaha/pytorch.cyclic.learning.rate/blob/master/cls.py
class CyclicLR(object):
    def __init__(self, optimizer, base_lr=1e-3, max_lr=6e-3,
                 step_size=2000, mode='triangular', gamma=1.,
                 scale_fn=None, scale_mode='cycle', last_batch_iteration=-1):

        if not isinstance(optimizer, Optimizer):
            raise TypeError('{} is not an Optimizer'.format(
                type(optimizer).__name__))
        self.optimizer = optimizer

        if isinstance(base_lr, list) or isinstance(base_lr, tuple):
            if len(base_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} base_lr, got {}".format(
                    len(optimizer.param_groups), len(base_lr)))
            self.base_lrs = list(base_lr)
        else:
            self.base_lrs = [base_lr] * len(optimizer.param_groups)

        if isinstance(max_lr, list) or isinstance(max_lr, tuple):
            if len(max_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} max_lr, got {}".format(
                    len(optimizer.param_groups), len(max_lr)))
            self.max_lrs = list(max_lr)
        else:
            self.max_lrs = [max_lr] * len(optimizer.param_groups)

        self.step_size = step_size

        if mode not in ['triangular', 'triangular2', 'exp_range'] \
                and scale_fn is None:
            raise ValueError('mode is invalid and scale_fn is None')

        self.mode = mode
        self.gamma = gamma

        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = self._triangular_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = self._triangular2_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = self._exp_range_scale_fn
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode

        self.batch_step(last_batch_iteration + 1)
        self.last_batch_iteration = last_batch_iteration

    def batch_step(self, batch_iteration=None):
        if batch_iteration is None:
            batch_iteration = self.last_batch_iteration + 1
        self.last_batch_iteration = batch_iteration
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

    def _triangular_scale_fn(self, x):
        return 1.

    def _triangular2_scale_fn(self, x):
        return 1 / (2. ** (x - 1))

    def _exp_range_scale_fn(self, x):
        return self.gamma**(x)

    def get_lr(self):
        step_size = float(self.step_size)
        cycle = np.floor(1 + self.last_batch_iteration / (2 * step_size))
        x = np.abs(self.last_batch_iteration / step_size - 2 * cycle + 1)

        lrs = []
        param_lrs = zip(self.optimizer.param_groups, self.base_lrs, self.max_lrs)
        for param_group, base_lr, max_lr in param_lrs:
            base_height = (max_lr - base_lr) * np.maximum(0, (1 - x))
            if self.scale_mode == 'cycle':
                lr = base_lr + base_height * self.scale_fn(cycle)
            else:
                lr = base_lr + base_height * self.scale_fn(self.last_batch_iteration)
            lrs.append(lr)
        return lrs


** Model Layers**

In [ ]:
Routings = 4 #5
Num_capsule = 5
Dim_capsule = 5#16
dropout_p = 0.25
LR = 0.001
T_epsilon = 1e-7
num_classes = 30


class Embed_Layer(nn.Module):
    def __init__(self, embedding_matrix=None, vocab_size=None, embedding_dim=300):
        super(Embed_Layer, self).__init__()
        self.encoder = nn.Embedding(vocab_size + 1, embedding_dim)
        if use_pretrained_embedding:
            # self.encoder.weight.data.copy_(t.from_numpy(np.load(embedding_path))) # 方法一，加载np.save的npy文件
            self.encoder.weight.data.copy_(t.from_numpy(embedding_matrix))  # 方法二

    def forward(self, x, dropout_p=0.25):
        return nn.Dropout(p=dropout_p)(self.encoder(x))


class GRU_Layer(nn.Module):
    def __init__(self):
        super(GRU_Layer, self).__init__()
        self.gru = nn.GRU(input_size=300,
                          hidden_size=hidden_size,
                          bidirectional=True)
        '''
        自己修改GRU里面的激活函数及加dropout和recurrent_dropout
        如果要使用，把rnn_revised import进来，但好像是使用cpu跑的，比较慢
       '''
        # # if you uncomment /*from rnn_revised import * */, uncomment following code aswell
        # self.gru = RNNHardSigmoid('GRU', input_size=300,
        #                           hidden_size=hidden_size,
        #                           bidirectional=True)

    # 这步很关键，需要像keras一样用glorot_uniform和orthogonal_uniform初始化参数
    def init_weights(self):
        ih = (param.data for name, param in self.named_parameters() if 'weight_ih' in name)
        hh = (param.data for name, param in self.named_parameters() if 'weight_hh' in name)
        b = (param.data for name, param in self.named_parameters() if 'bias' in name)
        for k in ih:
            nn.init.xavier_uniform_(k)
        for k in hh:
            nn.init.orthogonal_(k)
        for k in b:
            nn.init.constant_(k, 0)

    def forward(self, x):
        return self.gru(x)


# core caps_layer with squash func
class Caps_Layer(nn.Module):
    global hidden_size 
    def __init__(self, input_dim_capsule=hidden_size * 2, num_capsule=Num_capsule, dim_capsule=Dim_capsule, \
                 routings=Routings, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Caps_Layer, self).__init__(**kwargs)

        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size  # 暂时没用到
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = self.squash
        else:
            self.activation = nn.ReLU(inplace=True)

        if self.share_weights:
            self.W = nn.Parameter(
                nn.init.xavier_normal_(t.empty(1, input_dim_capsule, self.num_capsule * self.dim_capsule)))
        else:
            self.W = nn.Parameter(
                t.randn(BATCH_SIZE, input_dim_capsule, self.num_capsule * self.dim_capsule))  # 64即batch_size

    def forward(self, x):

        if self.share_weights:
            u_hat_vecs = t.matmul(x, self.W)
        else:
            print('add later')

        batch_size = x.size(0)
        input_num_capsule = x.size(1)
        u_hat_vecs = u_hat_vecs.view((batch_size, input_num_capsule,
                                      self.num_capsule, self.dim_capsule))
        u_hat_vecs = u_hat_vecs.permute(0, 2, 1, 3)  # 转成(batch_size,num_capsule,input_num_capsule,dim_capsule)
        b = t.zeros_like(u_hat_vecs[:, :, :, 0])  # (batch_size,num_capsule,input_num_capsule)

        for i in range(self.routings):
            b = b.permute(0, 2, 1)
            c = F.softmax(b, dim=2)
            c = c.permute(0, 2, 1)
            b = b.permute(0, 2, 1)
            outputs = self.activation(t.einsum('bij,bijk->bik', (c, u_hat_vecs)))  # batch matrix multiplication
            # outputs shape (batch_size, num_capsule, dim_capsule)
            if i < self.routings - 1:
                b = t.einsum('bik,bijk->bij', (outputs, u_hat_vecs))  # batch matrix multiplication
        return outputs  # (batch_size, num_capsule, dim_capsule)

    # text version of squash, slight different from original one
    def squash(self, x, axis=-1):
        s_squared_norm = (x ** 2).sum(axis, keepdim=True)
        scale = t.sqrt(s_squared_norm + T_epsilon)
        return x / scale
    
class Capsule_Main(nn.Module):
    def __init__(self, embedding_matrix=None, vocab_size=None):
        super(Capsule_Main, self).__init__()
        self.embed_layer = Embed_Layer(embedding_matrix, vocab_size)
        self.gru_layer = GRU_Layer()
        # 【重要】初始化GRU权重操作，这一步非常关键，acc上升到0.98，如果用默认的uniform初始化则acc一直在0.5左右
        self.gru_layer.init_weights()
        self.caps_layer = Caps_Layer()
        self.dense_layer = Dense_Layer()

    def forward(self, content):
        content1 = self.embed_layer(content)
        content2, _ = self.gru_layer(
            content1)  # 这个输出是个tuple，一个output(seq_len, batch_size, num_directions * hidden_size)，一个hn
        content3 = self.caps_layer(content2)
        output = self.dense_layer(content3)
        return output
    


**Attention - Keras Layer**

In [ ]:
class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)
        
        self.supports_masking = True

        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0
        
        weight = torch.zeros(feature_dim, 1)
        nn.init.xavier_uniform_(weight)
        self.weight = nn.Parameter(weight)
        
        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim))
        
    def forward(self, x, mask=None):
        feature_dim = self.feature_dim
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim), 
            self.weight
        ).view(-1, step_dim)
        
        if self.bias:
            eij = eij + self.b
            
        eij = torch.tanh(eij)
        a = torch.exp(eij)
        
        if mask is not None:
            a = a * mask

        a = a / torch.sum(a, 1, keepdim=True) + 1e-10

        weighted_input = x * torch.unsqueeze(a, -1)
        return torch.sum(weighted_input, 1)

** Neural Net - Embeddings - LSTM, GRU, Attention **

In [ ]:
class NeuralNet(nn.Module):
    global include_extra_features
    def __init__(self):
        
        super(NeuralNet, self).__init__()
  
        self.embedding = nn.Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1])
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        
        self.embedding_dropout = nn.Dropout2d(rate_drop_embedding)
        self.lstm = nn.LSTM(embedding_matrix.shape[1], hidden_size, bidirectional=True, batch_first=True)
        self.gru = nn.GRU(hidden_size*2, hidden_size, bidirectional=True, batch_first=True)
        
        self.lstm_attention = Attention(hidden_size*2, maxlen)
        self.gru_attention = Attention(hidden_size*2, maxlen)

        if (include_extra_features==True):
            self.linear = nn.Linear(hidden_size*8+features.shape[1], linear_size) #643:80 - 483:60 - 323:40
        else:
            self.linear = nn.Linear(hidden_size*8, linear_size) #643:80 - 483:60 - 323:40

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(rate_drop_linear)
        self.out = nn.Linear(linear_size, 1)
        
    def forward(self, x):
        if (include_extra_features==True):
            h_embedding = self.embedding(x[0])
        else:
            h_embedding = self.embedding(x) 
        #h_embedding = self.embedding(x)
        h_embedding = torch.squeeze(self.embedding_dropout(torch.unsqueeze(h_embedding, 0)))
        
        h_lstm, _ = self.lstm(h_embedding)
        h_gru, _ = self.gru(h_lstm)
        
        h_lstm_atten = self.lstm_attention(h_lstm)
        h_gru_atten = self.gru_attention(h_gru)
        
        avg_pool = torch.mean(h_gru, 1)
        max_pool, _ = torch.max(h_gru, 1)
        if (include_extra_features==True):
            f = torch.tensor(x[1], dtype=torch.float).cuda()
            conc = torch.cat((h_lstm_atten, h_gru_atten, avg_pool, max_pool,f), 1)
        else:
            conc = torch.cat((h_lstm_atten, h_gru_atten, avg_pool, max_pool), 1)
            
        conc = self.relu(self.linear(conc))
        conc = self.dropout(conc)
        out = self.out(conc)
        
        return out

** Neural Net 1 - Embedding, LSTM, GRU, Attention, Caps - Extra Features **

In [ ]:
class NeuralNet1(nn.Module):
    global include_extra_features
    
    def __init__(self):
        super(NeuralNet1, self).__init__()
        
        self.embedding = nn.Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1])
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = Trainable
        
        self.embedding_dropout = nn.Dropout2d(rate_drop_embedding)
        self.lstm = nn.LSTM( embedding_matrix.shape[1], hidden_size, bidirectional=True, batch_first=True)
        self.gru = nn.GRU(hidden_size * 2, hidden_size, bidirectional=True, batch_first=True)

        self.lstm2 = nn.LSTM(hidden_size * 2, hidden_size, bidirectional=True, batch_first=True)

        self.lstm_attention = Attention(hidden_size * 2, maxlen)
        self.gru_attention = Attention(hidden_size * 2, maxlen)
        
        if (include_extra_features==True):
            self.linear = nn.Linear(hidden_size*8+1+features.shape[1], linear_size) #643:80 - 483:60 - 323:40
        else:
            self.linear = nn.Linear(hidden_size*8+1, linear_size) #643:80 - 483:60 - 323:40
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(rate_drop_linear)
        self.fc = nn.Linear(linear_size**2,linear_size)
        self.out = nn.Linear(linear_size, 1)
        self.lincaps = nn.Linear(Num_capsule * Dim_capsule, 1)
        self.caps_layer = Caps_Layer()
    
    def forward(self, x):
        
#         Capsule(num_capsule=10, dim_capsule=10, routings=4, share_weights=True)(x)
        if (include_extra_features==True):
            h_embedding = self.embedding(x[0])
        else:
            h_embedding = self.embedding(x) 
            
        h_embedding = torch.squeeze(
            self.embedding_dropout(torch.unsqueeze(h_embedding, 0)))
        
        h_lstm, _ = self.lstm(h_embedding)
        h_gru, _ = self.gru(h_lstm)

        ##Attention Layer
        h_lstm_atten = self.lstm_attention(h_lstm)
        h_gru_atten = self.gru_attention(h_gru)
        
        # global average pooling
        avg_pool = torch.mean(h_gru, 1)
        # global max pooling
        max_pool, _ = torch.max(h_gru, 1)

     
        ##Capsule Layer        
        content3 = self.caps_layer(h_gru)
        content3 = self.dropout(content3)
        batch_size1 = content3.size(0)
        content3 = content3.view(batch_size1, -1)
        content3 = self.relu(self.lincaps(content3))        
        
        if (include_extra_features==True):
            f = torch.tensor(x[1], dtype=torch.float).cuda()
            conc = torch.cat((h_lstm_atten, h_gru_atten,content3, avg_pool, max_pool,f), 1)
        else:
            conc = torch.cat((h_lstm_atten, h_gru_atten,content3, avg_pool, max_pool), 1)
        conc = self.relu(self.linear(conc))
        conc = self.dropout(conc)

        out = self.out(conc)
        
        return out

**Neural Net 2 - Class CNN**

In [ ]:
class NeuralNet2(nn.Module):
    global include_extra_features
    def __init__(self, n_filters=100, filter_sizes=[3,4,5]):
        super(NeuralNet2,self).__init__()
        
        self.embedding = nn.Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1])
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = Trainable
        self.embedding_dropout = nn.Dropout2d(rate_drop_embedding)
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(fs,embedding_matrix.shape[1])) for fs in filter_sizes])
        if (include_extra_features==True):
            self.linear = nn.Linear(len(filter_sizes)*n_filters+ features.shape[1], 1)
        else:
            self.linear = nn.Linear(len(filter_sizes)*n_filters, 1)
        self.dropout = nn.Dropout(rate_drop_linear)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        #x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        
        if (include_extra_features==True):
            embedded = self.embedding(x[0])
        else:
            embedded = self.embedding(x) 
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        if (include_extra_features==True):
            f = torch.tensor(x[1], dtype=torch.float).cuda()
            conc = torch.cat(pooled, dim=1)
            conc = torch.cat((conc,f), 1)
        else:
            conc = torch.cat(pooled, dim=1)
            
        cat = self.dropout(conc)

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.linear(cat)

** Neural Net 3 - Embedding, LSTM, GRUs - Extra Features **

In [ ]:
class NeuralNet3(nn.Module):
    global include_extra_features
    def __init__(self):
        super(NeuralNet3, self).__init__()

        self.embedding = nn.Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1])
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = Trainable
              
        self.embedding_dropout = nn.Dropout2d(rate_drop_embedding)
        self.lstm = nn.LSTM(embedding_matrix.shape[1], hidden_size, bidirectional=True, batch_first=True)

        for name, param in self.lstm.named_parameters():
            if 'bias' in name:
                 nn.init.constant_(param, 0.0)
            elif 'weight_ih' in name:
                 nn.init.kaiming_normal_(param)
            elif 'weight_hh' in name:
                 nn.init.orthogonal_(param)

        self.gru = nn.GRU(hidden_size*2, hidden_size, bidirectional=True, batch_first=True)

        for name, param in self.gru.named_parameters():
            if 'bias' in name:
                 nn.init.constant_(param, 0.0)
            elif 'weight_ih' in name:
                 nn.init.kaiming_normal_(param)
            elif 'weight_hh' in name:
                 nn.init.orthogonal_(param)
        if (include_extra_features==True):
            self.linear = nn.Linear(hidden_size*6 + features.shape[1], linear_size)
        else:
            self.linear = nn.Linear(hidden_size*6 , linear_size)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(rate_drop_linear)
        self.out = nn.Linear(linear_size, 1)

    def forward(self, x):
        if (include_extra_features==True):
            h_embedding = self.embedding(x[0])
        else:
            h_embedding = self.embedding(x) 
        h_embedding = torch.squeeze(self.embedding_dropout(torch.unsqueeze(h_embedding, 0)))

        h_lstm, _ = self.lstm(h_embedding)

        h_gru, hh_gru = self.gru(h_lstm)
        hh_gru = hh_gru.view(-1, 2* hidden_size )

        avg_pool = torch.mean(h_gru, 1)
        max_pool, _ = torch.max(h_gru, 1)

        if (include_extra_features==True):
            f = torch.tensor(x[1], dtype=torch.float).cuda()
            conc = torch.cat(( hh_gru, avg_pool, max_pool,f), 1)
        else:
            conc = torch.cat(( hh_gru, avg_pool, max_pool), 1)

        conc = self.relu(self.linear(conc))
        conc = self.dropout(conc)
        out = self.out(conc)
        return out

** Neural Net 4 - Embedding, LSTM - Extra Features **

In [ ]:
class NeuralNet4(nn.Module):
    global include_extra_features
    def __init__(self, lstm_layer=2):
        super(NeuralNet4, self).__init__()
        
        self.embedding = nn.Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1])
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = Trainable             
        self.embedding_dropout = nn.Dropout2d(rate_drop_embedding)

        self.lstm = nn.LSTM(embedding_matrix.shape[1],
                            hidden_size,
                            num_layers=lstm_layer, 
                            dropout = rate_drop_hidden,
                            bidirectional=True,
                            batch_first=True)
        

        if (include_extra_features==True):
            self.linear = nn.Linear(hidden_size*lstm_layer*2+ features.shape[1], linear_size)
        else:
            self.linear = nn.Linear(hidden_size*lstm_layer*2 , linear_size)
         
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(rate_drop_linear)
        self.out = nn.Linear(linear_size, 1)
        

        
    def forward(self, x):

        if (include_extra_features==True):
            h_embedding = self.embedding(x[0])
        else:
            h_embedding = self.embedding(x) 

        h_embedding = torch.squeeze(self.embedding_dropout(torch.unsqueeze(h_embedding, 0)))

        lstm_out, (h_n, c_n) = self.lstm(h_embedding)

        conc= torch.cat([c_n[i,:, :] for i in range(c_n.shape[0])], dim=1)
        if (include_extra_features==True):
            f = torch.tensor(x[1], dtype=torch.float).cuda()
            conc = torch.cat((conc,f), dim=1)
           
        conc = self.relu(self.linear(conc))
        conc = self.dropout(conc)
        out = self.out(conc)
        return out
    



** Neural Net 5 - Embedding, LSTM, GRUs - Extra Features **

In [ ]:
class NeuralNet5(nn.Module):
    def __init__(self):
        super(NeuralNet5, self).__init__()

        self.embedding = nn.Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1])
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = Trainable             
        self.embedding_dropout = nn.Dropout2d(rate_drop_embedding)
        
        self.conv = nn.Conv1d(2*hidden_size, linear_size, 2)
        self.rnn = nn.GRU(embedding_matrix.shape[1], hidden_size, num_layers=1, bidirectional=True)
        if (include_extra_features==True):
            self.linear = nn.Linear(linear_size+ features.shape[1], 1)
        else:
            self.linear = nn.Linear(linear_size, 1)
        self.drop = nn.Dropout(p=rate_drop_linear)
        self.activation = nn.ReLU()

    def forward(self, x):
        
        if (include_extra_features==True):
            h_embedding = self.embedding(x[0])
        else:
            h_embedding = self.embedding(x) 
        h_embedding = self.embedding_dropout(h_embedding)
        h_embedding, hid = self.rnn(h_embedding)
        h_embedding = h_embedding.permute(0, 2, 1)

        cnn = self.conv(h_embedding)
        cnn = self.activation(cnn)
        cnn = cnn.permute(0, 2, 1)
        cnn = cnn.max(1)[0]
        
        if (include_extra_features==True):
            f = torch.tensor(x[1], dtype=torch.float).cuda()
            cnn = torch.cat((cnn,f), dim=1)
        x = self.linear(cnn)
        return x

    def get_trainable_parameters(self):
        return (param for param in self.parameters() if param.requires_grad)

In [ ]:
class NeuralNet6(nn.Module):
    global include_extra_features
    
    def __init__(self):
        super(NeuralNet6, self).__init__()
        
        self.embedding = nn.Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1])
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = Trainable
        
        self.embedding_dropout = nn.Dropout2d(rate_drop_embedding)
        self.lstm = nn.LSTM( embedding_matrix.shape[1], hidden_size, bidirectional=True, batch_first=True)
        self.gru = nn.GRU(hidden_size * 2, hidden_size, bidirectional=True, batch_first=True)

        self.lstm2 = nn.LSTM(hidden_size * 2, hidden_size, bidirectional=True, batch_first=True)

        self.lstm_attention = Attention(hidden_size * 2, maxlen)
        self.gru_attention = Attention(hidden_size * 2, maxlen)
        self.bn = nn.BatchNorm1d(16, momentum=0.5)
        if (include_extra_features==True):
            self.linear = nn.Linear(hidden_size*8+1+features.shape[1], linear_size) #643:80 - 483:60 - 323:40
        else:
            self.linear = nn.Linear(hidden_size*8+1, linear_size) #643:80 - 483:60 - 323:40
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(rate_drop_linear)
        self.fc = nn.Linear(linear_size**2,linear_size)
        self.out = nn.Linear(linear_size, 1)
        self.lincaps = nn.Linear(Num_capsule * Dim_capsule, 1)
        self.caps_layer = Caps_Layer()
    
    def forward(self, x):
        
#         Capsule(num_capsule=10, dim_capsule=10, routings=4, share_weights=True)(x)
        if (include_extra_features==True):
            h_embedding = self.embedding(x[0])
        else:
            h_embedding = self.embedding(x) 
            
        h_embedding = torch.squeeze(
            self.embedding_dropout(torch.unsqueeze(h_embedding, 0)))
        
        h_lstm, _ = self.lstm(h_embedding)
        h_gru, _ = self.gru(h_lstm)

        ##Attention Layer
        h_lstm_atten = self.lstm_attention(h_lstm)
        h_gru_atten = self.gru_attention(h_gru)
        
        # global average pooling
        avg_pool = torch.mean(h_gru, 1)
        # global max pooling
        max_pool, _ = torch.max(h_gru, 1)

     
        ##Capsule Layer        
        content3 = self.caps_layer(h_gru)
        content3 = self.dropout(content3)
        batch_size1 = content3.size(0)
        content3 = content3.view(batch_size1, -1)
        content3 = self.relu(self.lincaps(content3))        
        
        if (include_extra_features==True):
            f = torch.tensor(x[1], dtype=torch.float).cuda()
            conc = torch.cat((h_lstm_atten, h_gru_atten,content3, avg_pool, max_pool,f), 1)
        else:
            conc = torch.cat((h_lstm_atten, h_gru_atten,content3, avg_pool, max_pool), 1)
        conc = self.relu(self.linear(conc))
        conc = self.bn(conc)
        conc = self.dropout(conc)

        out = self.out(conc)
        
        return out
################


** Auxiliary Function to create Models **

In [ ]:
def create_model (label):
    
            if (label=="0"):
                return NeuralNet()
            if (label=="1"):
                return NeuralNet1()
            if (label=="2"):
                return NeuralNet2()
            if (label=="3"):
                return NeuralNet3()            
            if (label=="4"):
                return NeuralNet4()
            if (label=="5"):
                return NeuralNet5()
            if (label=="6"):
                return NeuralNet6()
            if (label=="7"):
                return create_model7()  
            if (label=="8"):
                return create_model8()
            if (label=="9"):
                return create_model9() 
            if (label=="10"):
                return create_model10()            
            if (label=="11"):
                return create_model11()                
            if (label=="12"):
                return create_model12()                
            if (label=="13"):
                return create_model13()                
            if (label=="14"):
                return create_model14()                
            if (label=="15"):
                return create_model15()                
            if (label=="16"):
                return create_model16()   
            if (label=="17"):
                return create_model17() 
            if (label=="18"):
                return create_model18() 
            if (label=="19"):
                return create_model19()
            if (label=="20"):
                return create_model20() 
            return None,"None"
            

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

** Best F1 - 2 ** Function to calculate F1 **

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve
import matplotlib.pyplot as plt
%matplotlib inline
def best_F12(y_true, y_proba, plot=False):
    precision, recall, thresholds = precision_recall_curve(y_true, y_proba)
    thresholds = np.append(thresholds, 1.001) 
    F = 2 / (1/precision + 1/recall)
    best_score = np.max(F)
    best_th = thresholds[np.argmax(F)]
    if plot:
        plt.plot(thresholds, F, '-b')
        plt.plot([best_th], [best_score], '*r')
        plt.show()

    search_result = {'threshold': best_th , 'f1': best_score}
    return best_th, best_score

** Best F1 - ** Function to calculate F1 **

In [ ]:
from scikitplot.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
def score_model (ypred ,ytrue,show_plot=False):
    bs=0
    bt=0
    for thresh in tqdm(np.arange(0.2, 0.402, 0.001)):
    #for thresh in tqdm([i * 0.01 for i in range(100)]):
        score = metrics.f1_score(ypred,(ytrue>=thresh).astype(int))
        if score >= bs:
            bt = thresh
            bs = score  
    rocauc= roc_auc_score(ypred,ytrue)
    precision, recall, thresholds = precision_recall_curve(ypred,ytrue)
    pr_auc = auc(recall, precision)
    if (show_plot):        
        print("\nBest F1 score at threshold %2.4f is %2.5f \n" % (bt, bs))
  
        print("\nROC AUC score is %2.5f \n" % rocauc)  

        print("\nPR AUC score is %2.5f \n" % pr_auc)
        plot_confusion_matrix(ypred, np.array(pd.Series(ytrue.reshape(-1,)).map(lambda x:1 if x>=bt else 0)))
        plt.show()
        d=confusion_matrix(ypred, (ytrue>=bt).astype(int))
        true_sincere=   d[0,:].sum()
        true_insincere= d[1,:].sum()
        pred_sincere=   d[:,0].sum()
        pred_insincere= d[:,1].sum()
        errors_insincere=d[0,1]
        errors_sincere=d[1,0]
        total_errors = errors_insincere+errors_sincere
        
        print ("Total True Label    :{:8d}".format( d.sum())," Insincere:{:8d}".format(true_insincere)," Sincere:{:8d}".format(true_sincere))
        print ("Total Predicted     :{:8d}".format( d.sum())," Insincere:{:8d}".format(pred_insincere)," Sincere:{:8d}".format(pred_sincere))
        print ("Errors              :{:8d}".format(total_errors)," Insincere:{:8d}".format( errors_insincere)," Sincere:{:8d}".format(errors_sincere))
        print ("Up/Down Diff        :          Insincere:{:8d}".format(true_insincere-pred_insincere)," Sincere:{:8d}".format(true_sincere-pred_sincere))
        print ("                               Insincere:({:5.2f}%)".format((errors_insincere/pred_insincere)*100)," Sincere:({:5.2f}%)".format((errors_sincere/pred_sincere) *100))
    return bt, bs , rocauc, pr_auc, precision.mean(), recall.mean()

'**Function to record results in a pandas dataframe**

In [ ]:
def rec_results (Model,Embedding,Thresh,Score,rocauc,prauc,prec,recall,StartTime):
    global resultsdf
    resultsdf = resultsdf.append({'Model':Model,
                                  'Method':method,
                                  'Embedding':Embedding,
                                  'Pretext':pretext_proc,
                                  'Weigth':wgt,
                                  'F1':Score,
                                  'Roc Auc':rocauc, 
                                  'PR Auc':prauc,
                                  'prec':prec,
                                  'recall':recall,
                                  "Threshold":Thresh,
                                  "Duration":str(round((time.time()-StartTime)/60,0)),
                                  'Epochs':epochs,
                                  'EpochsUsed':epoc_used,
                                  'Trainable':Trainable,
                                  'Opt':opt,
                                  'MaxLength':maxlen,
                                  'MaxVocab':max_vocab,                                  
                                  'batch_size':batch_size,
                                  'patience':patience,
                                  'hidden_size':hidden_size,
                                  'rate_drop_hidden':rate_drop_hidden,
                                  'linear_size':linear_size,
                                  'rate_drop_linear':rate_drop_linear,
                                  'rate_drop_embedding':rate_drop_embedding,
                                  'opt':opt,
                                  'sch':sch,
                                  'es_mon':es_mon,
                                  'es_mode':es_mode,
                                  'Date':datetime.now().strftime("%d-%m-%Y %H:%M")}, ignore_index=True)

**Function to load Embedding files (Glove, Paragram, Wiki and Google)**

In [ ]:
def load_embedding (emb,word_index) :

    estart = time.time()
    if debug:
            embedding_matrix = np.random.randn(10000,300)
            return embedding_matrix
    else:
            print('Indexing '+emb+' vectors')
            if (emb=="Glove"):
                EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
                def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
                embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
            if (emb=="Google"):
                EMBEDDING_FILE = '../input/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
                embeddings_index = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
            if (emb=="Paragram"):
                EMBEDDING_FILE =  '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt' 
                def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
                embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)
            if (emb=="Wiki"):
                EMBEDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'    
                embeddings_index = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=False)
            if (emb=="Quora"):
                EMBEDDING_FILE = "../quora_clean_300d.bin"    
                embeddings_index = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

            print("Vector",EMBEDDING_FILE )
            print("End Indexing:",(str(timedelta(seconds=(time.time()-estart)))) )
            estart = time.time()
            print('Preparing embedding matrix')

            out_of_features=0
            out_of_embedding=0
            in_embedding=0
            if ( emb=="Glove" or emb=="Paragram"):
                if (emb_out_of_vocab=="mean"):
                    all_embs = np.stack(embeddings_index.values())
                    emb_mean,emb_std = all_embs.mean(), all_embs.std()
                    embed_size = all_embs.shape[1]
 
                if (emb_out_of_vocab=="specific"):
                    if (emb=="Glove"):
                        all_embs = np.stack(embeddings_index.values())
                        emb_mean,emb_std = -0.005838499,0.48782197
                        embed_size = all_embs.shape[1]
                    if (emb=="Paragram"):
                        all_embs = np.stack(embeddings_index.values())
                        emb_mean,emb_std = -0.0053247833,0.49346462
                        embed_size = all_embs.shape[1]
                if (emb_out_of_vocab=="hybrid"):
                    if (emb=="Glove"):
                            all_embs = np.stack(embeddings_index.values())
                            emb_mean,emb_std = all_embs.mean(), all_embs.std()
                            embed_size = all_embs.shape[1]
                    if (emb=="Paragram"):
                        all_embs = np.stack(embeddings_index.values())
                        emb_mean,emb_std = -0.0053247833,0.49346462
                        embed_size = all_embs.shape[1]

                #word_index = tokenizer.word_index
                nb_words = min(max_vocab, len(word_index)+1)
                embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
                for word, i in word_index.items():
                    if i >= max_vocab: 
                        out_of_features+=1
                        continue
                    embedding_vector = embeddings_index.get(word)
                    if embedding_vector is not None: 
                        embedding_matrix[i] = embedding_vector
                        in_embedding+=1
                    else:
                        out_of_embedding+=1
            else: #"Google"
                embed_size=300
                #word_index = tokenizer.word_index
                nb_words = min(max_vocab, len(word_index)+1)
                embedding_matrix= (np.random.rand(nb_words, embed_size) - 0.5) / 5.0
                #embedding_matrix= (np.random.normal(nb_words, embed_size) - 0.5) / 5.0
                for word, i in word_index.items():
                    if i >= max_vocab:
                        out_of_features+=1
                        continue
                    if word in embeddings_index:
                        embedding_vector = embeddings_index.get_vector(word)
                        embedding_matrix[i] = embedding_vector
                        in_embedding+=1
                    else:
                        out_of_embedding+=1
            print("Total Vocabulary:",len(word_index))
            print("Out of Vocabulariy:", out_of_features)
            print ("Mapped Embeddings:",in_embedding)
            print ("Out of Embeddings:",out_of_embedding)
            print("End Preparing embedding matrix:",(str(timedelta(seconds=(time.time()-estart)))) )
            print ("-------------------------------------------")
            del embeddings_index
            return embedding_matrix

** Evaluate Model 0  - No Special Parameters**

** Evaluate Model 1 - with CyclicLR **

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

** Torch - Binary Acuracy **

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

** Torch Train **

In [ ]:

def torch_train(model, iterator, optimizer,criterion,scheduler, metric):
    
    epoch_loss = 0
    epoch_met = 0
    probs = []
    tgts = []
    model.train()
    
    #for batch in iterator:
    for i, (x_batch, y_batch) in enumerate(iterator):    
        optimizer.zero_grad()

        if (include_extra_features==True):
            f = kfold_X_features[i * batch_size:(i+1) * batch_size]
            predictions = model([x_batch,f])
        else:
            predictions = model(x_batch)
            
        if "CyclicLR" in str(scheduler) :
            scheduler.batch_step()
        
        loss = criterion(predictions, y_batch)
        if (metric=="val_acc"):
            emet = binary_accuracy(predictions, y_batch)
            epoch_met += emet.item()

        optimizer.zero_grad()           
        loss.backward()
        optimizer.step()
        proba= F.sigmoid(predictions).data.cpu().numpy()
        epoch_loss += loss.item()

        tgts.append(y_batch.data.cpu().numpy())
        probs.append(proba)
    tgts = np.vstack(tgts)
    probs = np.vstack(probs)
    if (metric=="val_acc"):
        met_result=epoch_met / len(iterator)
    if (metric=="val_auc"):
        met_result = np.mean(roc_auc_score(tgts, probs)) 
    if (metric=="val_f1"):
        bt ,met_result = best_F12(tgts, probs) 
    if (metric=="val_loss"):
        met_result=epoch_loss / len(iterator)
    return epoch_loss / len(iterator), met_result

** Torch Evaluate **

In [ ]:
def torch_evaluate(model, iterator, criterion, metric, results):
    
    epoch_loss = 0
    epoch_met = 0
    probs = []
    tgts = []
    model.eval()
    
    with torch.no_grad():
           
        for i, (x_batch, y_batch) in enumerate(iterator):
            if (include_extra_features):
                f =  kfold_X_valid_features[i * batch_size:(i+1) * batch_size]
                predictions = model([x_batch,f]).detach() 
            else:
                predictions = model(x_batch).detach()
                
            results[i * batch_size:(i+1) * batch_size] = sigmoid(predictions.cpu().numpy())[:, 0]  
            
            loss = criterion(predictions, y_batch)

            if (metric=="val_acc"):
                emet = binary_accuracy(predictions, y_batch)
                epoch_met += emet.item()
            epoch_loss += loss.item()
            proba= F.sigmoid(predictions).data.cpu().numpy()
            tgts.append(y_batch.data.cpu().numpy())
            probs.append(proba)
        tgts = np.vstack(tgts)
        probs = np.vstack(probs)
        if (metric=="val_acc"):
            met_result=epoch_met / len(iterator)
        if (metric=="val_auc"):
            met_result = np.mean(roc_auc_score(tgts, probs)) 
        if (metric=="val_f1"):
            bt ,met_result = best_F12(tgts, probs) 
        if (metric=="val_loss"):
            met_result=epoch_loss / len(iterator)
    return epoch_loss / len(iterator), met_result, probs

** Torch Predict **

In [ ]:
def torch_predict(model, iterator, criterion, extra_features, results):
      
    model.eval()
    
    with torch.no_grad():
    
        for i, (x_batch, ) in enumerate(iterator):
            if (include_extra_features):
                f = extra_features[i * batch_size:(i+1) * batch_size]            
                predictions = model([x_batch,f]).detach()               
            else:
                predictions = model(x_batch).detach()

            results[i * batch_size:(i+1) * batch_size] = sigmoid(predictions.cpu().numpy())[:, 0]
            
    return 

** Evaluate Model 2 - Utiliza TorchTrain, TorchEval, TorchPredic acima **

In [ ]:
def save_torch(m, info):
    torch.save(info, '../best_model.chck')
    torch.save(m, '../best_model.m')
    
def load_torch():
    m = torch.load('../best_model.m')
    info = torch.load('../best_model.chck')
    return m, info

** Evaluate Model  - Utiliza TorchTrain, TorchEval, TorchPredic acima **

In [ ]:
def evaluate_model (label,emb, embedding_matrix,train_X,train_y, t_X, t_y, v_X, v_y, epochs, batch_size, patience=0):
    
        global resultsdf 
        global met
        global clr
        global epoc_used 
        global kfold_X_features
        global kfold_X_valid_features
        
        best_thresh = 0.0
        best_score = 0.0
        epoc_used=0
        best_epoch=0
        mstart = time.time()        

        model=None
        model = create_model(label)
        mdln="Neural Net"+label
        print ("")
        print (mdln+'-'+emb)

        model.cuda()
        metric=es_mon
        
        step_size = 300
        base_lr, max_lr = 0.001, 0.003  
        if (opt=="rmsprop"):
            optimizer = torch.optim.RMSprop(model.parameters())

        if (opt=="adam2"):
            optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=max_lr)
            
        if (opt=="adam1"):
            optimizer = torch.optim.Adam(model.parameters())
        
        if (opt=="adam"):
            optimizer = optim.Adam(model.parameters(), lr=0.001,
                       betas=(0.9, 0.98), eps=1e-09)
        criterion = torch.nn.BCEWithLogitsLoss(reduction="sum")
        
        scheduler=None
        rscheduler=None
        
        if "lron" in sch:
            rscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.2, patience=patience, verbose=1)
        
        if "clr" in sch:

            ################################################################################################
            scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr, step_size=step_size, mode='exp_range', gamma=0.999999999)
            ###############################################################################################

        x_test_cuda = torch.tensor(test_X, dtype=torch.long).cuda()
        X_train_cuda= torch.tensor(train_X, dtype=torch.long).cuda()

        test = torch.utils.data.TensorDataset(x_test_cuda)
        train = torch.utils.data.TensorDataset(t_X, t_y)
        alltrain =torch.utils.data.TensorDataset(X_train_cuda)
        valid = torch.utils.data.TensorDataset(v_X, v_y)

        
        test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
        train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
        alltrain_loader = torch.utils.data.DataLoader(alltrain, batch_size=batch_size, shuffle=False)
        valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
        
        train_preds_fold = np.zeros(len(train_X))
        valid_preds_fold = np.zeros((v_X.size(0)))
        test_preds_fold = np.zeros(len(test_X))
        
        if (es_mode=="max"):
            best_metric=0
        else:
            best_metric =  float("inf") 
        valid_metrics=[0.]
        pt=0
        for epoch in range(epochs):
            
            start_time = time.time()
            train_loss, train_metric = torch_train(model, train_loader, optimizer, criterion, scheduler, metric)
            valid_loss, valid_metric , valid_proba= torch_evaluate(model, valid_loader, criterion, metric, valid_preds_fold)

            print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} |','Train '+es_mon+':',f' {train_metric*100:.2f}%',f' | Val. Loss: {valid_loss:.3f}',' | Val. '+es_mon+f': {valid_metric*100:.2f} |')
                
            valid_metrics += [valid_metric]

            if "lron" in sch:  ## Reduce on Plateu selected
                rscheduler.step(valid_loss)

            if "est" in sch:   ## Early Stop Selectec 
                if (es_mode=="max"):          
                    if (valid_metric > best_metric):
                        improved=True
                    else:
                        improved=False          
                else:      
                    if (valid_metric < best_metric):
                        improved=True
                    else:
                        improved=False
                epoc_used=epoch+1
                if (improved):       
                    
                    print('New best '+metric+':', valid_metric)
                    best_epoch=epoc_used
                    model_state_dict = model.state_dict()
                    checkpoint = {
                        'model': model_state_dict,
                        'optimizer': optimizer.state_dict(),
                        'epoch': epoch,
                         metric: valid_metric}
                    best_metric=valid_metric
                    save_torch (model,checkpoint )
                    #torch.save(checkpoint, '../'+model_name)
                    print('    - [Info] The checkpoint file has been updated.')
                else:
                    print("Metric not improved")
                    pt=pt+1
                    if (pt>patience):
                        break
        if "est" in sch:
            print ("Getting the Best Model - epoch ",best_epoch )
            model, checkpoint = load_torch()
        
        torch_predict(model, test_loader, criterion, test_features, test_preds_fold)

        ret_thresh,ret_f1,ret_rocauc, ret_prauc, ret_prec, ret_recall=score_model(v_y.cpu().numpy()[:, 0],valid_preds_fold, False)
        rec_results ("ST"+mdln,emb,ret_thresh,ret_f1,ret_rocauc, ret_prauc, ret_prec, ret_recall,mstart)
        if (Full_Train_Validation):
            torch_predict(model, alltrain_loader, criterion, features, train_preds_fold)
            
        import gc;           
        del model
        gc.collect()
        time.sleep(10)
        return  train_preds_fold,valid_preds_fold,test_preds_fold,ret_thresh,ret_f1,ret_rocauc, ret_prauc

** Stratified Kfold ** 

In [ ]:
def evaluate_stratifiedkfold (label,mt,emb,train_X,train_y, test_X, n_splits, stop_split, epochs, batch_size,patience, specific_split=None):
    
    global resultsdf 
    global mn
    global method
    global embedding_matrix
    global features
    global kfold_X_features
    global kfold_X_valid_features
    
#   train_preds = np.zeros((len(train_X)))
#    test_preds = np.zeros((len(test_X)))
#    features = np.array(features)
 
    method=mt
    train_stratified= np.zeros([len(train_X),(stop_split)])
    test_stratified = np.zeros([len(test_X),(stop_split)])
    val_stratified  = np.zeros([len(train_X)])
    
    msstart = time.time()
    
    embedding_matrix=None
    if  (emb=="Glove"):
        embedding_matrix=embedding_matrix_1
    if  (emb=="Wiki"):
        embedding_matrix=embedding_matrix_2
    if  (emb=="Paragram"):
        embedding_matrix=embedding_matrix_3
    if  (emb=="Google"):
        embedding_matrix=embedding_matrix_4
    if  (emb=="Quora"):
        embedding_matrix=embedding_matrix_5  
    if  (emb=="Combined"):
        embedding_matrix=embedding_matrix_combined
    if  (emb=="Combined2"):
        embedding_matrix=embedding_matrix_combined2    
    if  (emb=="Combined3"):
        embedding_matrix=embedding_matrix_combined3
    if  (emb=="Combined4"):
        embedding_matrix=embedding_matrix_combined4
    if  (emb=="Combined5"):
        embedding_matrix=embedding_matrix_combined5
    if  (emb=="Concatenated"):
        embedding_matrix=embedding_matrix_concatenated 
    if  (emb=="All"):
        embedding_matrix=embedding_matrix_all

    splits = list(StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED).split(train_X, train_y))
    mn=mn+1
    bf1s=np.zeros([stop_split])
    bt=np.zeros([stop_split])
    rocaucs= np.zeros([(stop_split)])
    praucs = np.zeros([(stop_split)])
    seed_torch(SEED) 
    for idx, (train_idx, valid_idx) in enumerate(splits):
        
            if (split_to_run > 0):
                if (idx != (split_to_run -1) ):
                    continue
                else:
                    idx=0
            if (include_extra_features):
                kfold_X_features = features[train_idx.astype(int)]
                kfold_X_valid_features = features[valid_idx.astype(int)]
            X_split = torch.tensor(train_X[train_idx], dtype=torch.long).cuda()
            y_split = torch.tensor(train_y[train_idx, np.newaxis], dtype=torch.float32).cuda()
            X_val = torch.tensor(train_X[valid_idx], dtype=torch.long).cuda()
            y_val = torch.tensor(train_y[valid_idx, np.newaxis], dtype=torch.float32).cuda()

            print(f'Fold {idx + 1}')

            pred_train_y, pred_val_y, pred_test_y, bt[idx] , bf1s[idx],rocaucs[idx], praucs[idx]=evaluate_model(md,emb,embedding_matrix,train_X,train_y, X_split, y_split, X_val, y_val, epochs, batch_size ,patience)
              
            val_stratified[valid_idx] =  pred_val_y
            train_stratified[:,idx]   = pred_train_y
            test_stratified[:,idx]    = pred_test_y
            if ((idx+1)==stop_split):
                break
    val_pred   =  val_stratified
    train_pred =  train_stratified.sum(axis=1)/stop_split
    test_pred  =  test_stratified.sum(axis=1)/stop_split
                         
    if (Full_Train_Validation==False):
        ret_thresh,ret_f1,ret_rocauc, ret_prauc, ret_prec, ret_recall=score_model(train_y,val_stratified, True)
        rec_results ("Model Kfold:"+label+"-skfold- "+str(n_splits)+"Stop: "+str(stop_split),emb,ret_thresh,ret_f1,ret_rocauc, ret_prauc, ret_prec, ret_recall,msstart)
        return train_pred,val_pred,test_pred,ret_thresh,ret_f1,ret_rocauc, ret_prauc
    else:
        ret_thresh, ret_f1,ret_rocauc, ret_prauc, ret_prec, ret_recall=score_model(train_y,train_pred, True)
        if (stop_split > 1):
            rec_results ("Model Mean:"+label+"-skfold- "+str(n_splits)+"Stop: "+str(stop_split),emb,bt.mean(),bf1s.mean(),rocaucs.mean(), praucs.mean(), ret_prec, ret_recall,msstart)
        return train_pred,val_pred,test_pred,bt.mean(),bf1s.mean(),rocaucs.mean(), praucs.mean()

** Defines which Embbedings to LOAD **

In [ ]:
load_Glove=False
load_Paragram=False
load_Quora=False
load_Wiki=False
load_Google=False
load_Combined=False
load_Combined2=False
load_Combined3=False
load_Combined4=False
load_Combined5=False
load_Concatenated=False
load_All=False
for ne,item in  enumerate(model_list): 
    emb=item[3] # Embeddings
    if emb=="Glove":
        load_Glove=True
    if emb=="Paragram":
        load_Paragram=True
    if emb=="Quora":
        load_Quora=True
    if emb=="Wiki":
        load_Wiki=True
    if emb=="Google":
        load_Google=True
    if emb=="Combined":
        load_Glove=True
        load_Paragram=True
        load_Combined=True
    if emb=="Combined2":
        load_Wiki=True
        load_Quora=True
        load_Combined2=True
    if emb=="Combined3":
        load_Glove=True
        load_Paragram=True
        load_Wiki=True
        load_Combined3=True
    if emb=="Combined4":
        load_Glove=True
        load_Paragram=True
        load_Quora=True
        load_Combined4=True
    if emb=="Combined5":
        load_Glove=True
        load_Paragram=True
        load_Wiki=True
        load_Google=True
        load_Combined5=True

    if emb=="Concatenated":
        load_Glove=True
        load_Paragram=True
        load_Concatenated=True
    if emb=="All":
        #load_Glove=True
        #load_Paragram=True
        #load_Google=True
        load_Wiki=True
        load_Quora=True
        load_All=True

** Load and Process Train,Val and Test  ** 

In [ ]:
###
###  Load Train,Val and Test 
###
tqdm_notebook().pandas()
train_X, test_X, train_y, test_df, word_index , features, test_features = load_and_process(pretext_proc)

** Dimensionality Reductions - Embeddings **

In [ ]:
# Build a Latent Semantic Indexing Model
from sklearn.decomposition import TruncatedSVD

def reduce_matrix (emb_mat,n=10):
  svd_model = TruncatedSVD(n_components=n)
  reduced = svd_model.fit_transform(emb_mat)
  return reduced 

**Load Embedding Matrix from Glove,Wki,Paragram and Google. **
** Create Combination or Concatenatedf Matrix **

In [ ]:
###
### Load Embedding Matrix
###
seed_torch(SEED)
if load_Glove:
    embedding_matrix_1 = load_embedding("Glove",word_index)
if load_Wiki:
    embedding_matrix_2 = load_embedding("Wiki",word_index)
if load_Paragram:
    embedding_matrix_3 = load_embedding("Paragram",word_index)
if load_Google:
    embedding_matrix_4 = load_embedding("Google",word_index)
if load_Quora:
    embedding_matrix_5 = load_embedding("Quora",word_index)
## Simple average: http://aclweb.org/anthology/N18-2031

# We have presented an argument for averaging as
# a valid meta-embedding technique, and found experimental
# performance to be close to, or in some cases 
# better than that of concatenation, with the
# additional benefit of reduced dimensionality  


## Unweighted DME in https://arxiv.org/pdf/1804.07983.pdf

# “The downside of concatenating embeddings and 
#  giving that as input to an RNN encoder, however,
#  is that the network then quickly becomes inefficient
#  as we combine more and more embeddings.”
  
#embedding_matrix = np.mean([embedding_matrix_1, embedding_matrix_2, embedding_matrix_3], axis = 0)
if load_Combined: 
    embedding_matrix_combined = np.mean([embedding_matrix_1, embedding_matrix_3], axis = 0)
    print ("Combined Matrix Dimension:",np.shape(embedding_matrix_combined))

if load_Combined2: 
    embedding_matrix_combined2 = np.mean([embedding_matrix_2, embedding_matrix_5], axis = 0)
    print ("Combined Matrix Dimension:",np.shape(embedding_matrix_combined2))    
    
if load_Combined3: 
    embedding_matrix_combined3 = np.mean([embedding_matrix_1, embedding_matrix_2,embedding_matrix_3], axis = 0)
    print ("Combined Matrix Dimension:",np.shape(embedding_matrix_combined3))
    
if load_Combined4: 
    embedding_matrix_combined4 = np.mean([embedding_matrix_1, embedding_matrix_3,embedding_matrix_5], axis = 0)
    print ("Combined Matrix Dimension:",np.shape(embedding_matrix_combined4))
    
if load_Combined5: 
    embedding_matrix_combined5 = np.mean([embedding_matrix_1, embedding_matrix_2,embedding_matrix_3,embedding_matrix_4], axis = 0)
    print ("Combined Matrix Dimension:",np.shape(embedding_matrix_combined5))
    
if load_Concatenated:
    embedding_matrix_concatenated = np.concatenate((embedding_matrix_1, embedding_matrix_3), axis=1)  
    #del embedding_matrix_1, embedding_matrix_2, embedding_matrix_3, embedding_matrix_4
    #gc.collect()
    print ("Concatenates Matrix dimensions",np.shape(embedding_matrix_concatenated))
    
if load_All:
    #e1=reduce_matrix (embedding_matrix_1,n=150)
    e2=reduce_matrix (embedding_matrix_2,n=100)
    #e3=reduce_matrix (embedding_matrix_3,n=150)
    #e4=reduce_matrix (embedding_matrix_4,n=50)
    e5=reduce_matrix (embedding_matrix_5,n=100)
    embedding_matrix_all = np.concatenate((e2,e5), axis=1) 
    #del embedding_matrix_1
    del embedding_matrix_2
    #del embedding_matrix_3
    #del embedding_matrix_4
    del embedding_matrix_5
    print ("All SVD Matrix dimensions",np.shape(embedding_matrix_all))

**Run Models
**

In [ ]:
###
### Run All Models
###
resultsdf = pd.DataFrame(columns=['Model','F1','Roc Auc',"PR Auc",'Embedding','Pretext','Weigth','Duration','Method',
                                  'Epochs','EpochsUsed','patience','batch_size'
                                  ,'MaxLength','MaxVocab' ])


### Matrix of Predictions - Validation and Test
### Stores All Predicted Values

allpred_train_y=np.zeros([len(train_X),(len(model_list))])
allpred_val_y=np.zeros([len(train_X),(len(model_list))])
allpred_test_y=np.zeros([len(test_X),(len(model_list))])
allbf1=np.zeros([(len(model_list))])
allrocauc=np.zeros([(len(model_list))])
allprauc=np.zeros([(len(model_list))])
allthresh=np.zeros([(len(model_list))])

epoc_used=0
mn=-1 
print ("Max Vocab",max_vocab)
#model_list.append(["18",1,"5","Combine",8,pat,bs,splits,stop,nh,nd,ndh,ndd,nds,loss_bin,opt_rmsprop,met_acc,mon_acc,mode_acc])
for ne,item in  enumerate(model_list): 
    md=item[0]  # Model to Run
    wgt=item[1]    # Model weigth for essembling
    mt=item[2]   # Model Evaluation Method
    emb=item[3] # Embeddings
    epochs=item[4] # Number of Epochs
    patience=item[5] # Patience - Wait patience epochs if score does not improve
    batch_size=item[6] # Batch Size to Fit Model
    nsplits=item[7]   # Number of Splits for Stratified k-fold
    stop_split=item[8]    # Number of Splits to run in Stratified k-fold
    split_to_run=item[9]
    hidden_size=item[10]  # Size of hidden layers in models (lstm and gru)
    linear_size=item[11] # Size of hidden layers in model - dense
    rate_drop_hidden = item[12] # lstm and gru - drop-out rates
    rate_drop_linear = item[13] # dense drop-out rates
    rate_drop_embedding=item[14]  # spatial drop-out rate
    loss=item[15] # Model Loss
    opt=item[16]  #Models Optmizer
    sch=item[17]  #Models Metric
    es_mon=item[18] # Early stop monitor 
    es_mode=item[19]  # Early stop mode
    if (nsplits != stop_split):
        Full_Train_Validation=True
    else:
        Full_Train_Validataion=False

    print ("******Evaluating Model:"+md+" Evaluation Method:",mt)
    trainp,valp,testp,bt,bf1,brocauc,bprauc=evaluate_stratifiedkfold (md,mt,emb,train_X,train_y, test_X, nsplits, stop_split, epochs, batch_size,patience)
    allbf1[mn]=bf1
    allthresh[mn]=bt
    allrocauc[mn]=brocauc
    allprauc[mn]=bprauc
    allpred_train_y[:,mn] = trainp
    allpred_val_y[:,mn] = valp
    allpred_test_y[:,mn] = testp
    print ("****************************************************")



**Calculate weights and apply to results****Calculate weights and apply to results**

In [ ]:
#### Applying Weights to results
#### Manual/Score
twgt=0
wgt_test_y=0
wgt_train_y=0
wgt_val_y=0
essemblet="Essemble: "
for ne,item in  enumerate(model_list):
    if (Weight_Method=="Manual"):
        wg=item[1]
    if (Weight_Method=="f1"):
        wg=allbf1[ne]
    if (Weight_Method=="auc"):
        wg=allrocauc[ne]
    twgt=twgt+wg
    wgt_val_y=wgt_val_y+(allpred_val_y[:,ne]*wg)
    wgt_train_y=wgt_train_y+(allpred_train_y[:,ne]*wg)
    wgt_test_y =wgt_test_y +(allpred_test_y[:,ne]*wg)
    essemblet=essemblet+item[0]+" "
wgt_val_y  =wgt_val_y/twgt
wgt_train_y=wgt_train_y/twgt
wgt_test_y =wgt_test_y/twgt

**Write Output to submission file**

In [ ]:
def bestThresshold(y_train,train_preds):
    tmp = [0,0,0] # idx, cur, max
    delta = 0
    for tmp[0] in tqdm(np.arange(0.2, 0.402, 0.001)):
        tmp[1] = f1_score(y_train, np.array(train_preds)>tmp[0])
        if tmp[1] > tmp[2]:
            delta = tmp[0]
            tmp[2] = tmp[1]
    print('best threshold is {:.4f} with F1 score: {:.5f}'.format(delta, tmp[2]))
    return delta

if (Full_Train_Validation):
        print ("Full Validation")
        best_thresh,best_score=best_F12(train_y,wgt_train_y,True )
        print ("Threshold Valid F12:",best_thresh)
        delta = bestThresshold(train_y,wgt_train_y)
        print ("Threshold Valid Delta:",delta)
        best_thresh,best_score,rauc,prauc,mp,mr=score_model(train_y,wgt_train_y,True )
        rec_results (essemblet+"FULL Validation ","-",best_thresh,best_score,rauc,prauc,mp,mr,bstart) 
        print ("Threshold Full Valid:",best_thresh)
else:
        print ("Stratified Cross Validation")
        best_thresh,best_score=best_F12(train_y,wgt_val_y,True )
        print ("Threshold Valid F12:",best_thresh)
        delta = bestThresshold(train_y,wgt_val_y)
        print ("Threshold Valid Delta:",delta)
        best_thresh,best_score,rauc,prauc,mp,mr=score_model(train_y,wgt_val_y,True )
        rec_results (essemblet+"CV Validation ","-",best_thresh,best_score,rauc,prauc,mp,mr,bstart)
        print ("Threshold Valid F1:",best_thresh)

array_test_y=np.array(wgt_test_y)

print ("")
print (allthresh)
print ("Threshold Minimo:",allthresh.min(),(np.count_nonzero(array_test_y >= allthresh.min())))
print ("Threshold Médio:",allthresh.mean(),(np.count_nonzero(array_test_y >= allthresh.mean())))
print ("Threshold Maximo:",allthresh.max(),(np.count_nonzero(array_test_y >= allthresh.max())))
if (debug):
    tg=allthresh.min()
    itg=tg
    ni=(np.count_nonzero(array_test_y >= tg))
    ii=ni
    print ("tg i",itg," Insincere i:",ii)
    while  (ni >= 3488) :
            tg=tg+0.001
            ni=(np.count_nonzero(array_test_y >= tg) )
    print ("tg",tg,tg-itg," Insincere:",ni, ii-ni)

# Write the output
#best_thresh
#y_te = (array_test_y >= 0.2940).astype(np.int)
y_te = (array_test_y >= best_thresh).astype(np.int)
submit_df = pd.DataFrame({"qid": test_df["qid"], "prediction": y_te})
submit_df.to_csv("./submission.csv",index=False)
print ("Submit Rows:",submit_df["qid"].count())
#rec_results ('Time Elapsed','',0,0,bstart)
display (resultsdf)